In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from data_extraction.dummy_data_extractor import extract_dummy_data

def prphet_forecasting(df, length_of_missing_data, data_logging_interval, dqStart):

df = extract_dummy_data("dummy_data")
df = df.at[1,"his"]
df.reset_index(inplace=True)
df = df.dropna()


# Rename columns for convenience
data.columns = ['ts', 'temp', 'new_point']

# # Convert 'ts' column to datetime and handle timezone offset
# data['ts'] = pd.to_datetime(data['ts'].replace(' Dubai', ''), errors='coerce')
# Drop rows where datetime parsing failed
data = data.dropna(subset=['ts'])

# # Clean temperature column and convert to numeric
# data['temp'] = data['temp'].str.replace('°C', '').astype(float)

# Separate data for temperature
df_temp = data[['ts', 'temp']].rename(columns={'ts': 'ds', 'temp': 'y'})

# Ensure 'ds' column is timezone-naive
df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)

# Splitting data into train and test sets
train_size = int(len(df_temp) * 0.8)  # 80% for training, 20% for testing
train_data = df_temp.iloc[:train_size]
test_data = df_temp.iloc[train_size:]

samples = (length_of_missing_data / data_logging_interval) + 1

# Initialize Prophet model with tuned hyperparameters for temperature
model_temp = Prophet(seasonality_mode='additive',
                     interval_width=0.95,
                     changepoint_prior_scale=0.001)

# Fit the model on training data
model_temp.fit(train_data)

# Create future DataFrame for temperature (next 200 samples, assuming 5-minute intervals)
future_temp = model_temp.make_future_dataframe(periods=samples, freq='5T')

# Predict the future values for temperature
forecast_temp = model_temp.predict(future_temp)

# Compute RMSE for temperature on test data
actual_temp = test_data['y'].values
predicted_temp = forecast_temp['yhat'].values[-len(test_data):]  # Last len(test_data) predictions
rmse_temp = np.sqrt(mean_squared_error(actual_temp, predicted_temp))
print(f"RMSE for temp: {rmse_temp}")

return predicted_temp